In [1]:
import os
import pandas as pd
import numpy as np
import s3fs
from datetime import datetime
from datetime import timedelta
import calendar
import boto3
import io
import ast

## Load the data

In [3]:
sample_sub_df = pd.read_csv('data/sample_submission_2_adj.csv').drop(["Unnamed: 0"], axis = 1)
sample_sub_df.head(1)

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02


In [9]:
# Read labels
label_1_df = pd.read_csv("data/label_1.csv", header = None)\
               .rename(columns = {0:"user_id_hash", 
                                  1:"user_purchase_binary_7_days"})
label_2_df = pd.read_csv("data/label_2.csv", header = None)\
               .rename(columns = {0:"user_id_hash", 
                                  1:"user_purchase_binary_14_days"})
label_1_df.head(1)

,user_id_hash,user_purchase_binary_7_days
0,bf676611754201cc93ca1e2bcce8ca2c7ff0105186ebc0...,1.0


### Data Dictionary for Total
0 - user_id  
1 - total number of sessions  
2 - total number of events  
3 - median number of events  
4 - total sum of purchases  
5 - total amount of purchases  
6 - median session duration

In [10]:
# Read Total
total_train_df = pd.read_csv("data/train_total.csv", header = None)\
                   .rename(columns = {0:"user_id_hash", 
                                      1:"num_sessions",
                                      2:"num_events",
                                      3:"median_num_events",
                                      4:"num_purchases",
                                      5:"amt_purchases",
                                      6:"median_session_duration"})
total_test_df = pd.read_csv("data/test_total.csv", header = None)\
                  .rename(columns = {0:"user_id_hash", 
                                     1:"num_sessions",
                                     2:"num_events",
                                     3:"median_num_events",
                                     4:"num_purchases",
                                     5:"amt_purchases",
                                     6:"median_session_duration"})
total_train_df.head(1)

,user_id_hash,num_sessions,num_events,median_num_events,num_purchases,amt_purchases,median_session_duration
0,0d261313961125c71f330a8a8d59857bc48fefd0b4278c...,1,35,35.0,0,0.0,1626343.0


In [147]:
# Read Last 7 Days
train_7_df = pd.read_csv("feature_eng/train_last_7_days.csv", header = None)\
                   .rename(columns = {0:"user_id_hash", 
                                      1:"num_sessions_7",
                                      2:"num_events_7",
                                      3:"median_num_events_7",
                                      4:"num_purchases_7",
                                      5:"amt_purchases_7",
                                      6:"median_session_duration_7"})
test_7_df = pd.read_csv("feature_eng/test_last_7_days.csv", header = None)\
                  .rename(columns = {0:"user_id_hash", 
                                     1:"num_sessions_7",
                                     2:"num_events_7",
                                     3:"median_num_events_7",
                                     4:"num_purchases_7",
                                     5:"amt_purchases_7",
                                     6:"median_session_duration_7"})
train_7_df.head(1)

,user_id_hash,num_sessions_7,num_events_7,median_num_events_7,num_purchases_7,amt_purchases_7,median_session_duration_7
0,c7385007b6f158e913411d1563d1f5ef90ea003050b105...,1,1,1.0,0,0.0,0.0


In [158]:
# Read Last 14 Days
train_14_df = pd.read_csv("feature_eng/train_last_14_days.csv", header = None)\
                   .rename(columns = {0:"user_id_hash", 
                                      1:"num_sessions_14",
                                      2:"num_events_14",
                                      3:"median_num_events_14",
                                      4:"num_purchases_14",
                                      5:"amt_purchases_14",
                                      6:"median_session_duration_14"})
test_14_df = pd.read_csv("feature_eng/test_last_14_days.csv", header = None)\
                  .rename(columns = {0:"user_id_hash", 
                                     1:"num_sessions_14",
                                     2:"num_events_14",
                                     3:"median_num_events_14",
                                     4:"num_purchases_14",
                                     5:"amt_purchases_14",
                                     6:"median_session_duration_14"})
train_14_df.head(1)

,user_id_hash,num_sessions_14,num_events_14,median_num_events_14,num_purchases_14,amt_purchases_14,median_session_duration_14
0,a7d68ea7dcf1bc9fc2d455c19d1ba409951609735a543b...,1,1,1.0,0,0.0,0.0


### Data Dictionary for Sessions
0 - user_id  
1 - date_user_created   
2 - most_freq_country  
3 - most_freq_os  
4 - num_uniq_device_id

In [165]:
# Read Sessions
sessions_df = pd.read_csv("feature_eng/sessions.csv", header = None)\
                   .rename(columns = {0:"user_id_hash", 
                                      1:"date_user_created",
                                      2:"most_freq_country",
                                      3:"most_freq_os",
                                      4:"num_uniq_device_id"})\
                   .dropna()
sessions_df.head(1)

,user_id_hash,date_user_created,most_freq_country,most_freq_os,num_uniq_device_id
0,46d54b4fab292c461cdf8f3825f7106907b300b2d28cf3...,2018-10-19 22:04:58,US,iOS,1


## Feature Engineering

In [200]:
train_fe = pd.merge(total_train_df, sessions_df, how = "left").dropna()
train_fe = pd.merge(train_fe, train_7_df, how = "left").fillna(0)
train_fe = pd.merge(train_fe, train_14_df, how = "left").fillna(0)
test_fe = pd.merge(total_test_df, sessions_df, how = "left").dropna()
test_fe = pd.merge(test_fe, test_7_df, how = "left").fillna(0)
test_fe = pd.merge(test_fe, test_14_df, how = "left").fillna(0)

In [201]:
# Check null values
for column in train_fe.columns:
    if train_fe[column].isnull().any():
        print('{0} has {1} null values'.format(column, train_fe[column].isnull().sum()))
for column in test_fe.columns:
    if test_fe[column].isnull().any():
        print('{0} has {1} null values'.format(column, test_fe[column].isnull().sum()))

### Create column for Number of Days Existed

In [202]:
# Create column for Number of Days Existed
# For training, compute days between first date and Dec.2
# For testing, compute days between first date and Dec.16
train_fe["num_days_existed"] = [(datetime(2018, 12, 2) - datetime.strptime(date, '%Y-%m-%d %H:%M:%S')).days 
                                for date in train_fe.date_user_created.values]
test_fe["num_days_existed"] = [(datetime(2018, 12, 16) - datetime.strptime(date, '%Y-%m-%d %H:%M:%S')).days 
                                for date in test_fe.date_user_created.values]

### Create column for Days Since Most Recent Session

In [203]:
most_recent_train_df = pd.read_csv('feature_eng/most_recent_session_train.csv', header = None)\
                         .rename(columns = {0:"user_id_hash", 1:"most_recent_session"})
most_recent_test_df = pd.read_csv('feature_eng/most_recent_session_test.csv', header = None)\
                        .rename(columns = {0:"user_id_hash", 1:"most_recent_session"})

In [204]:
train_fe_adj = pd.merge(train_fe, most_recent_train_df, how = 'left')
test_fe_adj = pd.merge(test_fe, most_recent_test_df, how = 'left')
train_fe_adj2 = train_fe_adj
test_fe_adj2 = test_fe_adj
train_fe_adj2["most_recent_session"] = train_fe_adj["most_recent_session"].fillna(train_fe_adj["date_user_created"])
test_fe_adj2["most_recent_session"] = test_fe_adj["most_recent_session"].fillna(test_fe_adj["date_user_created"])

In [205]:
train_fe["days_since_last_session"] = [(datetime(2018, 12, 2) - datetime.strptime(date, '%Y-%m-%d %H:%M:%S')).days 
                                        for date in train_fe_adj2.most_recent_session.values]
test_fe["days_since_last_session"] = [(datetime(2018, 12, 16) - datetime.strptime(date, '%Y-%m-%d %H:%M:%S')).days 
                                        for date in test_fe_adj2.most_recent_session.values]

### Create column for frequent OS

In [206]:
train_fe["most_freq_os_android"] = [1 if x == 'Android OS' else 0 for x in train_fe.most_freq_os.values]
train_fe["most_freq_os_ios"] = [1 if x != 'Android OS' else 0 for x in train_fe.most_freq_os.values]
test_fe["most_freq_os_android"] = [1 if x == 'Android OS' else 0 for x in test_fe.most_freq_os.values]
test_fe["most_freq_os_ios"] = [1 if x != 'Android OS' else 0 for x in test_fe.most_freq_os.values]

### Create dummies regarding regions

In [207]:
country_code_df = pd.read_csv("country_code.csv", header = None)
country_code_df = country_code_df.loc[1:,[1,5,6]]
country_code_df = country_code_df[country_code_df[1] != 'AQ'].dropna()
country_code_dict = {}
country_code_dict['ZZ'] = "Unknown"
country_code_dict['XK'] = "Southern Europe"
country_code_dict['NAM'] = "Sub-Saharan Africa"
for country_code in country_code_df[1].values:
    country_code_dict[country_code] = country_code_df[country_code_df[1]==country_code][6].values[0]

In [208]:
train_fe["most_freq_region"] = [country_code_dict[x] for x in train_fe.most_freq_country.values]
test_fe["most_freq_region"] = [country_code_dict[x] for x in test_fe.most_freq_country.values]

In [209]:
region_list = []
for region in set(train_fe.most_freq_region.values):
    if list(train_fe.most_freq_region.values).count(region) > 20000:
        print(region, list(train_fe.most_freq_region.values).count(region))
        region_list.append(region)

Western Europe 21095
Sub-Saharan Africa 23872
South-eastern Asia 55423
Northern America 297832
Latin America and the Caribbean 24213
Australia and New Zealand 20900
Southern Asia 57538
Northern Europe 60393


In [210]:
for region in region_list:
    col_name = f"is_{region.replace(' ','_').lower()}"
    train_fe[col_name] = [1 if x == region else 0 for x in train_fe.most_freq_region.values]
    test_fe[col_name] = [1 if x == region else 0 for x in test_fe.most_freq_region.values]

In [211]:
train_fe.head(1)

,user_id_hash,num_sessions,num_events,median_num_events,num_purchases,amt_purchases,median_session_duration,date_user_created,most_freq_country,most_freq_os,...,most_freq_os_ios,most_freq_region,is_western_europe,is_sub-saharan_africa,is_south-eastern_asia,is_northern_america,is_latin_america_and_the_caribbean,is_australia_and_new_zealand,is_southern_asia,is_northern_europe
0,0d261313961125c71f330a8a8d59857bc48fefd0b4278c...,1,35,35.0,0,0.0,1626343.0,2018-10-13 07:33:07,ID,iOS,...,1,South-eastern Asia,0,0,1,0,0,0,0,0


### Create Labels

In [212]:
train_fe = pd.merge(train_fe, label_1_df, how = 'left').fillna(0)
train_fe = pd.merge(train_fe, label_2_df, how = 'left').fillna(0)

## Machine Learning

In [1392]:
train_ml = train_fe.copy()
test_ml = test_fe.copy()

In [1393]:
# Check if the labels are correct. The expected outcome is {0.0}.
set(train_ml[train_ml.user_purchase_binary_7_days != train_ml.user_purchase_binary_14_days]\
 .user_purchase_binary_7_days)

{0.0}

In [1394]:
train_ml.columns

Index(['user_id_hash', 'num_sessions', 'num_events', 'median_num_events',
       'num_purchases', 'amt_purchases', 'median_session_duration',
       'date_user_created', 'most_freq_country', 'most_freq_os',
       'num_uniq_device_id', 'num_sessions_7', 'num_events_7',
       'median_num_events_7', 'num_purchases_7', 'amt_purchases_7',
       'median_session_duration_7', 'num_sessions_14', 'num_events_14',
       'median_num_events_14', 'num_purchases_14', 'amt_purchases_14',
       'median_session_duration_14', 'num_days_existed',
       'days_since_last_session', 'most_freq_os_android', 'most_freq_os_ios',
       'most_freq_region', 'is_western_europe', 'is_sub-saharan_africa',
       'is_south-eastern_asia', 'is_northern_america',
       'is_latin_america_and_the_caribbean', 'is_australia_and_new_zealand',
       'is_southern_asia', 'is_northern_europe', 'user_purchase_binary_7_days',
       'user_purchase_binary_14_days'],
      dtype='object')

In [1395]:
train_ml["is_purchase_7"] = [1.0 if y > 0 else 0.0 for y in train_ml.num_purchases_7]
train_ml["is_purchase_ever"] = [1.0 if y > 0 else 0.0 for y in train_ml.num_purchases]
train_ml["is_session_7"] = [1.0 if y > 0 else 0.0 for y in train_ml.num_sessions_7]
train_ml["is_purchase_14"] = [1.0 if y > 0 else 0.0 for y in train_ml.num_purchases_14]
train_ml["is_session_14"] = [1.0 if y > 0 else 0.0 for y in train_ml.num_sessions_14]
train_ml['purchase_per_day'] = train_ml.num_purchases/train_ml.num_days_existed
test_ml["is_purchase_7"] = [1.0 if y > 0 else 0.0 for y in test_ml.num_purchases_7]
test_ml["is_purchase_ever"] = [1.0 if y > 0 else 0.0 for y in test_ml.num_purchases]
test_ml["is_session_7"] = [1.0 if y > 0 else 0.0 for y in test_ml.num_sessions_7]
test_ml["is_purchase_14"] = [1.0 if y > 0 else 0.0 for y in test_ml.num_purchases_14]
test_ml["is_session_14"] = [1.0 if y > 0 else 0.0 for y in test_ml.num_sessions_14]
test_ml['purchase_per_day'] = test_ml.num_purchases/test_ml.num_days_existed

In [1396]:
# per num_days_existed
train_ml["num_uniq_session_db_num_days_existed"] = train_ml.num_sessions / train_ml.num_days_existed
test_ml["num_uniq_session_db_num_days_existed"] = test_ml.num_sessions / test_ml.num_days_existed
train_ml["total_purchase_amt_db_num_days_existed"] = train_ml.amt_purchases / train_ml.num_days_existed
test_ml["total_purchase_amt_db_num_days_existed"] = test_ml.amt_purchases / test_ml.num_days_existed
# per num_uniq_session_id
train_ml["total_purchase_amt_db_num_uniq_session_id"] = train_ml.amt_purchases / train_ml.num_sessions
test_ml["total_purchase_amt_db_num_uniq_session_id"] = test_ml.amt_purchases / test_ml.num_sessions
train_ml["total_purchase_amt_db_total_num_purchase"] = train_ml.amt_purchases / train_ml.num_purchases
test_ml["total_purchase_amt_db_total_num_purchase"] = test_ml.amt_purchases / test_ml.num_purchases
train_ml = train_ml.fillna(0)
test_ml = test_ml.fillna(0)

In [1560]:
train_ml["amt_purchase_past_wk_per_purchase"] = train_ml.amt_purchases_7 / train_ml.num_purchases_7
test_ml["amt_purchase_past_wk_per_purchase"] = test_ml.amt_purchases_7 / test_ml.num_purchases_7
train_ml["amt_purchase_past_wk_per_session"] = train_ml.amt_purchases_7 / train_ml.num_sessions_7
test_ml["amt_purchase_past_wk_per_session"] = test_ml.amt_purchases_7 / test_ml.num_sessions_7
train_ml = train_ml.fillna(0)
test_ml = test_ml.fillna(0)

In [1685]:
# Set labels
labels_1 = train_ml.user_purchase_binary_7_days
labels_2 = train_ml.user_purchase_binary_14_days
features_train = train_ml[['purchase_per_day', 'days_since_last_session', 
                           'is_purchase_ever', 'is_session_7',
                           'num_purchases_7', 'most_freq_os_android',
                           'is_western_europe', 'is_sub-saharan_africa',
                           'is_south-eastern_asia', 'is_northern_america',
                           'is_latin_america_and_the_caribbean', 'is_australia_and_new_zealand',
                           'is_southern_asia', 'is_northern_europe',
                           "num_uniq_session_db_num_days_existed",
                           "total_purchase_amt_db_num_days_existed",
                           "total_purchase_amt_db_num_uniq_session_id",
                           "total_purchase_amt_db_total_num_purchase",
                           "amt_purchase_past_wk_per_purchase",
                           "amt_purchase_past_wk_per_session"
                          ]]
features_test = test_ml[['purchase_per_day', 'days_since_last_session', 
                         'is_purchase_ever', 'is_session_7',
                         'num_purchases_7', 'most_freq_os_android',
                         'is_western_europe', 'is_sub-saharan_africa',
                         'is_south-eastern_asia', 'is_northern_america',
                         'is_latin_america_and_the_caribbean', 'is_australia_and_new_zealand',
                         'is_southern_asia', 'is_northern_europe',
                         "num_uniq_session_db_num_days_existed",
                         "total_purchase_amt_db_num_days_existed",
                         "total_purchase_amt_db_num_uniq_session_id",
                         "total_purchase_amt_db_total_num_purchase",
                         "amt_purchase_past_wk_per_purchase",
                         "amt_purchase_past_wk_per_session"
                        ]]

In [1686]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF_Factor"] = [variance_inflation_factor(features_train.values, i) for i in range(features_train.shape[1])]
vif["features"] = features_train.columns
vif.sort_values(by = 'VIF_Factor', ascending=False)

,VIF_Factor,features
0,5.682662,purchase_per_day
15,4.673646,total_purchase_amt_db_num_days_existed
1,3.677264,days_since_last_session
9,2.879877,is_northern_america
5,2.800305,most_freq_os_android
17,2.718239,total_purchase_amt_db_total_num_purchase
2,2.313933,is_purchase_ever
4,2.306654,num_purchases_7
19,2.107003,amt_purchase_past_wk_per_session
3,2.000386,is_session_7


In [1563]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

In [1564]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize

## Label 1

In [1803]:
# Resampling
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
rus = RandomUnderSampler()
features_res_1, labels_res_1 = rus.fit_sample(features_train, labels_1)
print(f"Resampled dataset shape: {Counter(labels_res_1)}")

Resampled dataset shape: Counter({0.0: 4396, 1.0: 4396})


In [1804]:
from sklearn.model_selection import train_test_split
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(features_res_1, labels_res_1, test_size=0.3)

In [1808]:
import xgboost as xgb
xgb_pars_1 = {'min_child_weight': 1, 'eta': 0.001, 'colsample_bytree': 0.7, 'max_depth': 6,
            'lambda': 0.0, 'n_estimators' : 9,
            'subsample': 0.3, 'booster' : 'gblinear', 'silent': 1,
            'eval_metric': 'auc', 'objective': 'binary:logistic', 'scale_pos_weight': 1}

In [1809]:
dtrain_1 = xgb.DMatrix(X_train_1, label=y_train_1)
dvalid_1 = xgb.DMatrix(X_val_1, label=y_val_1)
watchlist = [(dtrain_1, 'train'), (dvalid_1, 'valid')]

In [1810]:
gbm_1 = xgb.train(xgb_pars_1, dtrain_1, 50, watchlist,
                maximize=False, verbose_eval=10)

[0]	train-auc:0.963129	valid-auc:0.970395
[10]	train-auc:0.96313	valid-auc:0.970428
[20]	train-auc:0.963119	valid-auc:0.970433
[30]	train-auc:0.963127	valid-auc:0.970448
[40]	train-auc:0.963111	valid-auc:0.970456
[49]	train-auc:0.963099	valid-auc:0.970454


In [1858]:
pred_1 = gbm_1.predict(dvalid_1)
roc_auc_score(y_val_1, pred_1)

0.9704536057693691

## Label 2

In [1750]:
# Resampling
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
rus = RandomUnderSampler()
features_res_2, labels_res_2 = rus.fit_sample(features_train, labels_2)
print(f"Resampled dataset shape: {Counter(labels_res_2)}")

Resampled dataset shape: Counter({0.0: 5553, 1.0: 5553})


In [1751]:
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(features_res_2, labels_res_2, test_size=0.3)

In [1780]:
xgb_pars_2 = {'min_child_weight': 1, 'eta': 0.001, 'colsample_bytree': 0.7, 'max_depth': 6,
              'lambda': 0, 'n_estimators': 9, 
              'subsample': 0.4, 'booster' : 'gblinear', 'silent': 1,
              'eval_metric': 'auc', 'objective': 'binary:logistic', 'scale_pos_weight': 1}

In [1781]:
dtrain_2 = xgb.DMatrix(X_train_2, label=y_train_2)
dvalid_2 = xgb.DMatrix(X_val_2, label=y_val_2)
watchlist_2 = [(dtrain_2, 'train'), (dvalid_2, 'valid')]

In [1782]:
gbm_2 = xgb.train(xgb_pars_2, dtrain_2, 50, watchlist_2,
                maximize=False, verbose_eval=10)

[0]	train-auc:0.95693	valid-auc:0.963155
[10]	train-auc:0.956946	valid-auc:0.963172
[20]	train-auc:0.956964	valid-auc:0.963179
[30]	train-auc:0.956962	valid-auc:0.963192
[40]	train-auc:0.956968	valid-auc:0.963211
[49]	train-auc:0.956968	valid-auc:0.963213


In [1812]:
pred_2 = gbm_2.predict(dvalid_2)
roc_auc_score(y_val_2, pred_2)

0.963212841737521

### Build result data frame

In [1814]:
# For resampling
_, num_cols = features_test.shape
new_cols_name_test = [f'f{i}' for i in range(num_cols)]
old_cols_name_test = list(features_test.columns)
features_test_adj = features_test.copy()
for j in range(num_cols):
    features_test_adj = features_test_adj.rename(columns = {old_cols_name_test[j]:new_cols_name_test[j]})
features_test_adj.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19'],
      dtype='object')

In [1815]:
result_df = pd.DataFrame()
result_df["user_id_hash"] = test_ml.user_id_hash.values
dtest_1 = xgb.DMatrix(features_test_adj)
dtest_2 = xgb.DMatrix(features_test_adj)
result_df['user_purchase_binary_7_days'] = gbm_1.predict(dtest_1)
result_df['user_purchase_binary_14_days'] = gbm_2.predict(dtest_2)
result_df.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,c39d635765ff135752a20fea4283e35edb76f5cd0babe1...,0.459621,0.460405
1,0d261313961125c71f330a8a8d59857bc48fefd0b4278c...,0.449424,0.450537
2,a7d68ea7dcf1bc9fc2d455c19d1ba409951609735a543b...,0.479725,0.480784
3,fcb42eb24a4a769cd511a0dc6d6d7cd3f79cc1bfef0098...,0.463018,0.463971
4,fa2d2e99027a7099d3ab22b686fc2d8d03404be93931b8...,0.460067,0.461010


In [1816]:
len(result_df)

619422

In [1817]:
sample_sub_df_adj = pd.DataFrame()
sample_sub_df_adj["user_id_hash"] = sample_sub_df.user_id_hash.values
len(sample_sub_df_adj)

312568

In [1818]:
final_result = pd.merge(sample_sub_df_adj, result_df, how = "left").fillna(0.0)
final_result.isnull().sum() # Check if there is nan

user_id_hash                    0
user_purchase_binary_7_days     0
user_purchase_binary_14_days    0
dtype: int64

In [1821]:
final_result.to_csv('submission/sample_submission_fatpapaya_xgb_13.csv', index = False)